In [6]:
!head -10 ml-1m/ratings.dat


1::1193::5::978300760
1::661::3::978302109
1::914::3::978301968
1::3408::4::978300275
1::2355::5::978824291
1::1197::3::978302268
1::1287::5::978302039
1::2804::5::978300719
1::594::4::978302268
1::919::4::978301368


In [7]:
import pandas as pd
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter="::", engine="python", header=None)

In [8]:
ratings.columns = ["UserID::MovieID::Rating::Timestamp".split("::")]
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
pd.read_csv?

In [9]:
ratings.isnull().sum()

UserID       0
MovieID      0
Rating       0
Timestamp    0
dtype: int64

In [10]:
import numpy as np
np.unique(ratings['Rating'])

array([1, 2, 3, 4, 5])

* Do we have any duplicate ratings, i.e., a (userid, movieid) combination that is repeated more than once?

In [11]:
user_movie_pairs = zip(ratings['UserID'].values, ratings['MovieID'].values)

In [12]:
print len(user_movie_pairs), len(set(user_movie_pairs))

1000209 1000209


In [21]:
print ratings['UserID'].unique()
set(ratings['UserID'].unique()) == set(range(1, 6041))
# User ids are range(1, 6041), i.e., are all values between 1 and 6040
# We can use a simple mapping: user_index=UserID-1

[   1    2    3 ..., 6038 6039 6040]


True

In [32]:
!head ml-1m/movies.dat

1::Toy Story (1995)::Animation|Children's|Comedy
2::Jumanji (1995)::Adventure|Children's|Fantasy
3::Grumpier Old Men (1995)::Comedy|Romance
4::Waiting to Exhale (1995)::Comedy|Drama
5::Father of the Bride Part II (1995)::Comedy
6::Heat (1995)::Action|Crime|Thriller
7::Sabrina (1995)::Comedy|Romance
8::Tom and Huck (1995)::Adventure|Children's
9::Sudden Death (1995)::Action
10::GoldenEye (1995)::Action|Adventure|Thriller


In [46]:
len(movies.MovieID) == len(movies.MovieID.unique())

True

In [57]:
movies = pd.read_csv('ml-1m/movies.dat', delimiter='::', engine='python', header=None)
movies.columns = 'MovieID::Title::Genres'.split('::')
movies.head()

print "The following movie id's are missing from movie table"
print sorted(set(range(1, 3953)) - set(movies.MovieID))
print 

mi = ratings['MovieID'].unique()
mi.sort()
print sorted(set(range(1, 3953)) - set(mi)) 
# movie id have some missing values in addition to the missing values above
# , i.e., there are movies that are not rated by any user.

movie_index_to_ID = dict(zip(range(movies.MovieID.unique().shape[0]), movies.MovieID))
#movie_ID_to_index = dict(zip(movies.MovieID, range(movies.MovieID.unique().shape[0])))
movie_ID_to_index = {k: v for v, k in movie_index_to_ID.iteritems()}

print movie_index_to_ID.items()[-10:]
print movie_ID_to_index.items()[-10:]


The following movie id's are missing from movie table
[91, 221, 323, 622, 646, 677, 686, 689, 740, 817, 883, 995, 1048, 1072, 1074, 1182, 1195, 1229, 1239, 1338, 1402, 1403, 1418, 1435, 1451, 1452, 1469, 1478, 1481, 1491, 1492, 1505, 1506, 1512, 1521, 1530, 1536, 1540, 1560, 1576, 1607, 1618, 1634, 1637, 1638, 1691, 1700, 1712, 1736, 1737, 1745, 1751, 1761, 1763, 1766, 1775, 1778, 1786, 1790, 1800, 1802, 1803, 1808, 1813, 1818, 1823, 1828, 1838, 3815]

[51, 91, 109, 115, 143, 221, 284, 285, 323, 395, 399, 400, 403, 604, 620, 622, 625, 629, 636, 646, 654, 675, 676, 677, 683, 686, 689, 693, 699, 713, 721, 723, 727, 738, 739, 740, 752, 768, 770, 772, 773, 777, 794, 795, 797, 812, 816, 817, 819, 822, 825, 845, 855, 856, 857, 871, 873, 883, 890, 894, 979, 983, 995, 1001, 1045, 1048, 1052, 1065, 1072, 1074, 1075, 1106, 1108, 1109, 1110, 1122, 1137, 1140, 1141, 1143, 1146, 1155, 1156, 1157, 1158, 1159, 1166, 1182, 1195, 1229, 1239, 1308, 1309, 1314, 1318, 1319, 1338, 1368, 1400, 1402, 1403, 1

In [9]:
from scipy.sparse import csr_matrix
row = ratings['MovieID'].values
col = ratings['UserID'].values
data = ratings['Rating'].values
ratings_array = csr_matrix((data, (row, col)))



In [10]:
ratings_array.shape

(3953, 6041)

In [11]:
np.sum(ratings_array != 0)

1000209

In [12]:
import numpy as np
#X = np.array([[1,1], [2, 1], [3, 1.2], [4, 1], [5, 0.8], [6, 1]])
from sklearn.decomposition import NMF
model = NMF(n_components=5, init='random', random_state=0)
W = model.fit_transform(ratings_array) 
H = model.components_



In [13]:
H.shape


(5, 6041)

In [14]:
model.n_components_

5

In [15]:
X = ratings_array

In [16]:
def trace_dot(X, Y):
    """Trace of np.dot(X, Y.T)."""
    return np.dot(X.ravel(), Y.ravel())
norm_X = np.dot(X.data, X.data)
norm_WH = trace_dot(np.dot(np.dot(W.T, W), H), H)
cross_prod = trace_dot((X * H.T), W)
error = np.sqrt(norm_X + norm_WH - 2. * cross_prod)

In [17]:
error/X.shape[0]/X.shape[1]

0.00012841350131919519

In [ ]:
reload(MF)

In [ ]:
MF.calculate_learning_curve(ratings.values[:100, :120])

In [1]:

from pyspark import SparkConf, SparkContext
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

conf = SparkConf() \
        .setAppName("MovieLensALS") \
        .set("spark.executor.memory", "1g")
sc = SparkContext(conf=conf)


In [3]:

sc.setLogLevel("WARN")

def parse_data(line):
    fields = line.split("::")
    tup =  (int(fields[0]), int(fields[1]), float(fields[2]), int(fields[3][-1]))
    return tup


In [49]:
from sklearn import linear_model, datasets

# import some data to play with
iris = datasets.load_iris()
X = iris.data[:, :2]  # we only take the first two features.
Y = iris.target




In [51]:
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [54]:
from pyspark.ml.tuning import CrossValidator
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder



lr = LogisticRegression(maxIter=10)

paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, np.logspace(-2, 2, 5)) \
    .build()

cnt = 0

class Evaluator:
    def evaluate(self, dataset):
        global cnt
        cnt += 1
        print dataset.count()
        print cnt
        #print dataset['features'].collect()
        #print dataset['label']
        print
        return 1
    
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          #evaluator=BinaryClassificationEvaluator(),
                          evaluator=Evaluator(),
                          numFolds=3)  # use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(training)



33
1

33
2

33
3

33
4

33
5

28
6

28
7

28
8

28
9

28
10

39
11

39
12

39
13

39
14

39
15



AttributeError: Evaluator instance has no attribute 'isLargerBetter'

In [24]:
lr.params

[Param(parent=u'LogisticRegression_493d9d098c33c544652c', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'),
 Param(parent=u'LogisticRegression_493d9d098c33c544652c', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'),
 Param(parent=u'LogisticRegression_493d9d098c33c544652c', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'),
 Param(parent=u'LogisticRegression_493d9d098c33c544652c', name='featuresCol', doc='features column name.'),
 Param(parent=u'LogisticRegression_493d9d098c33c544652c', name='fitIntercept', doc='whether to fit an intercept term.'),
 Param(parent=u'LogisticRegression_493d9d098c33c544652c', name='labelCol', doc='label column name.'),
 Param(parent=u'LogisticRegression_493d9d098c33c544652c', name='maxIter', doc='max number of i

In [9]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()


In [13]:
import os
spark_home = os.environ['SPARK_HOME']
training = spark.read.format("libsvm").load(os.path.join(spark_home, "data/mllib/sample_libsvm_data.txt"))


/usr/local/spark


In [52]:
training

DataFrame[label: double, features: vector]